In [ ]:
!wget -O /kaggle/working/dataset.zip "https://my.microsoftpersonalcontent.com/personal/802fb67c5dc88f59/_layouts/15/download.aspx?UniqueId=5dc88f59-b67c-202f-8080-f72e00000000&Translate=false&tempauth=v1e.eyJzaXRlaWQiOiJmNmNlZWYyYi1jNzQ2LTQxZjUtOWQ5MC1mOWQzZmM3N2MxODMiLCJhdWQiOiIwMDAwMDAwMy0wMDAwLTBmZjEtY2UwMC0wMDAwMDAwMDAwMDAvbXkubWljcm9zb2Z0cGVyc29uYWxjb250ZW50LmNvbUA5MTg4MDQwZC02YzY3LTRjNWItYjExMi0zNmEzMDRiNjZkYWQiLCJleHAiOiIxNzMxMjc3NTQ1In0.E_mXbyPcs-EyrGMRBGJ3pyrWdsWOqcN4-h4bI9ASxkXG6AeX9nQx_IPOcJi78udKu5ZQtT3fnP9MT2J0ZLau-uB4iq4_UOTDiN2mROjAk3vTaGArh8_ZjT8viUZKJSloaDsMJCdx7V4xOMvoJCpxm3tXY0hNmcJQQ3s8B4aWW-tGi97gWNl582imIRGcmdXPNBDQJrD6BCQ6ZwVFeKli4qkWKK8iJuwN7xGsfSDAdaUGQSZFGdp3PhctY2h9o4vTmP88BVnzhIMGmGe7UYTJwzZnGllQ_3_80Fmco5f8xyTaryVIajmFmcrsuOuXNrnWcIKdeoD7uppZeUIAI2U9ijdUvXBuIZIbtf4LkWuOM9c5XRgAkG8ndGEIM2Qz5vsZZsQ9HZKvP8a-OVAGHkx9p0PBhl2V_k5lGMGYN4ojyXRQSSwqUUSr1PoI8JIwFBlY52n__55yzR3QhpUZVMUAUIGz7tTkeGuGGT2NcA1eQBacL55ftapugsqymBAxdc0DL8i84eh8dRiMFyGW_x6UQA.kG9R6Qe9KUoiEfK6nH_YkCNi4QMcLrtL27nOvwKuD3g&ApiVersion=2.0"

In [ ]:
import zipfile
import os

# Define paths
input_zip_path = '/kaggle/working/dataset.zip'  # Update this path if needed
output_folder = '/kaggle/working/dataset'

# Create the output directory if it doesn’t exist
os.makedirs(output_folder, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print("Dataset unzipped successfully.")


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
import os

# Define a function to normalize image data
def normalize_images(images):
    return images / 255.0  # Normalize RGB values to [0, 1]

# Define a function to standardize fMRI data
def standardize_fmri(fmri_data):
    mean = np.mean(fmri_data, axis=0)
    std = np.std(fmri_data, axis=0)
    return (fmri_data - mean) / std  # Standardize to mean 0 and std 1

# Main function to preprocess data
def preprocess_and_save(subject_num, raw_data_dir, preprocessed_data_dir):
    # Load image data
    image_data = np.load(os.path.join(raw_data_dir, 'images_256.npz'))
    train_images = image_data['train_images']
    test_images = image_data['test_images']

    # Normalize images
    train_images_normalized = normalize_images(train_images)
    test_images_normalized = normalize_images(test_images)

    # Load fMRI data for the subject
    subject_data = np.load(os.path.join(raw_data_dir, f'sbj_{subject_num}.npz'))
    train_fmri = subject_data['arr_0']
    test_fmri = subject_data['arr_2']

    # Standardize fMRI data
    train_fmri_standardized = standardize_fmri(train_fmri)
    test_fmri_standardized = standardize_fmri(test_fmri)

    # Save the preprocessed data as .npz files in the preprocessed data directory
    np.savez(os.path.join(preprocessed_data_dir, f'preprocessed_data_subject_{subject_num}_train.npz'),
             images=train_images_normalized, fmri=train_fmri_standardized)
    np.savez(os.path.join(preprocessed_data_dir, f'preprocessed_data_subject_{subject_num}_test.npz'),
             images=test_images_normalized, fmri=test_fmri_standardized)

    print(f"Preprocessed data for Subject {subject_num} saved as .npz files.")

# Set paths for Kaggle
raw_data_dir = '/kaggle/working/dataset/data/Kamitani/npz'
preprocessed_data_dir = '/kaggle/working/preprocessed'

# Ensure the output directory exists
os.makedirs(preprocessed_data_dir, exist_ok=True)

# Preprocess and save data for each subject
for subject_num in range(1, 6):
    preprocess_and_save(subject_num, raw_data_dir, preprocessed_data_dir)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define dataset and model as before
class FMRIImageDataset(Dataset):
    def __init__(self, npz_file):
        data = np.load(npz_file)
        self.images = data['images']
        self.fmri = data['fmri']
       
    def __len__(self):
        return len(self.images)
       
    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float32)
        fmri = torch.tensor(self.fmri[idx], dtype=torch.float32)
        return fmri, image

In [ ]:
# Generator Network with reduced layer sizes
class Generator(nn.Module):
    def __init__(self, fmri_size, image_size):
        super(Generator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(fmri_size, 256),  # Reduced layer size
            nn.ReLU(),
            nn.Linear(256, 512),       # Reduced layer size
            nn.ReLU(),
            nn.Linear(512, 1024),      # Reduced layer size
            nn.ReLU(),
            nn.Linear(1024, image_size),
            nn.Tanh()  # Output range [-1, 1]
        )

    def forward(self, x):
        return self.network(x)

# Discriminator Network with reduced layer sizes
class Discriminator(nn.Module):
    def __init__(self, image_size):
        super(Discriminator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(image_size, 512),       # Reduced layer size
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),              # Reduced layer size
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),              # Reduced layer size
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.network(x)


In [ ]:
# Set up device and parallel processing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(fmri_size=4466, image_size=256 * 256 * 3).to(device)
discriminator = Discriminator(image_size=256 * 256 * 3).to(device)

if torch.cuda.device_count() > 1:
    generator = nn.DataParallel(generator)
    discriminator = nn.DataParallel(discriminator)

# Optimizers and loss function
criterion = nn.BCEWithLogitsLoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
scaler = GradScaler()

# Load preprocessed dataset
train_dataset = FMRIImageDataset('/kaggle/working/preprocessed/preprocessed_data_subject_1_train.npz')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [ ]:
import torch
import torch.optim as optim
from torch.amp import autocast, GradScaler

# Environment setting to reduce memory fragmentation
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Hyperparameters
fmri_size = 4466
image_size = 256 * 256 * 3
batch_size = 2  # Reduced to fit memory constraints
num_epochs = 10
learning_rate = 0.0002
accumulation_steps = 4

# Initialize models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(fmri_size=fmri_size, image_size=image_size).to(device)
discriminator = Discriminator(image_size=image_size).to(device)

if torch.cuda.device_count() > 1:
    generator = nn.DataParallel(generator)
    discriminator = nn.DataParallel(discriminator)

# Optimizers and loss function
criterion = nn.BCEWithLogitsLoss()
optimizer_g = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
scaler = GradScaler()

# Load dataset
train_dataset = FMRIImageDataset('/kaggle/working/preprocessed/preprocessed_data_subject_1_train.npz')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop with gradient accumulation and mixed precision
for epoch in range(num_epochs):
    for i, (fmri, real_images) in enumerate(train_loader):
        fmri = fmri.view(fmri.size(0), -1).to(device)
        real_images = real_images.view(real_images.size(0), -1).to(device)
        real_labels = torch.ones(fmri.size(0), 1).to(device)
        fake_labels = torch.zeros(fmri.size(0), 1).to(device)

        # Train Discriminator
        with autocast('cuda'):
            outputs = discriminator(real_images)
            d_loss_real = criterion(outputs, real_labels)

            fake_images = generator(fmri)
            outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)

            d_loss = (d_loss_real + d_loss_fake) / accumulation_steps

        optimizer_d.zero_grad()
        scaler.scale(d_loss).backward()
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer_d)
            scaler.update()

        # Train Generator
        with autocast('cuda'):
            fake_images = generator(fmri)
            outputs = discriminator(fake_images)
            g_loss = criterion(outputs, real_labels) / accumulation_steps

        optimizer_g.zero_grad()
        scaler.scale(g_loss).backward()
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer_g)
            scaler.update()

        # Clear cache
        torch.cuda.empty_cache()

        # Monitor losses and scores
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")


In [ ]:
torch.save(generator.module.state_dict() if isinstance(generator, nn.DataParallel) else generator.state_dict(),
           '/kaggle/working/generator_model.pth')


In [ ]:
def generate_and_visualize(generator, test_loader, device):
    generator.eval()
    with torch.no_grad():
        for fmri, original_images in test_loader:
            fmri = fmri.view(fmri.size(0), -1).to(device)
            generated_images = generator(fmri).cpu().view(-1, 256, 256, 3).numpy()
            original_images = original_images.cpu().view(-1, 256, 256, 3).numpy()

            generated_images = (generated_images + 1) / 2
            original_images = (original_images + 1) / 2

            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            axes[0].imshow(original_images[0])
            axes[0].set_title("Original Image")
            axes[0].axis("off")

            axes[1].imshow(generated_images[0])
            axes[1].set_title("Generated Image")
            axes[1].axis("off")
            
            plt.show()
            break

test_loader = DataLoader(FMRIImageDataset('/kaggle/working/preprocessed/preprocessed_data_subject_1_test.npz'), batch_size=1)
generate_and_visualize(generator, test_loader, device)
